<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/C3TR_Adapter_Japanese_English_Translation_sample_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日英・英日翻訳サンプルコード(Japanese-English English-Japanese Translation sample code)


C3TR-Adapterを使って日本語を英語、英語を翻訳するサンプルコードです。  
Sample code to translate Japanese to English and English to Japanese using C3TR-Adapter.

上段メニューの「ランタイム」→「すべてのセルを実行」で実行してください  
Please execute it by clicking "Runtime" -> "Execute All Cells" in the upper menu  


もしくは各セルの[ ]の部分をクリックして実行してください  
Or click on the [ ] part of each cell to execute it.  


In [1]:
%%bash
pip install peft==0.9.0 bitsandbytes==0.42.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 27.7 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

model_id = "unsloth/gemma-7b-bnb-4bit"
peft_model_id = "webbigdata/C3TR-Adapter"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
model = PeftModel.from_pretrained(model = model, model_id = peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/800M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [3]:
def trans(my_str):
    input_ids = tokenizer(my_str, return_tensors="pt",
        padding=True, max_length=1600, truncation=True).input_ids.cuda()

    # Translation
    generated_ids = model.generate(input_ids=input_ids,
        max_new_tokens=800,
        use_cache=True,
        prompt_lookup_num_tokens=10
        )
    full_outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return full_outputs[0].split("### Answer:\n")[-1].strip()

In [4]:
ret = trans("""
### Instructions:
Translate English to Japanese within the context of subculture.
### Input:
Female employee: Hi Boss...
Boss: Yes?.......
Female employee: I'm pregnant.
Boss: ....
Female employee: So...
Female employee: I'd like to take few days leave to talk to my boyfriend on this; please.
Boss: First of all...pls come talk to my wife. Just now she read your message... Why don't you type all in one text?
### Answer:
""")
print(ret)

ret = trans("""
### Instructions:
Translate Japanese to English.
### Input:
■グローバリゼーションとは何か

問い：21世紀、世界はグローバリゼーションの世の中になった。
ではグローバリゼーションとはいったい何のことだろうか。
答え：アメリカの子どもの将来の夢がサッカー選手で、
イタリアの子どもの夢がマンガ家で、
日本の子どもの夢がメジャーリーガーだということ。

### Answer:
""")
print(ret)

女性社員：こんにちは上司さん…
上司：はい？……
女性社員：妊娠しました。
上司：……
女性社員：ですから…
女性社員：彼氏と話し合うために、数日間休暇をもらいたいのですが。どうかお願いいたします。
上司：まずまず…妻に話してください。今彼女がメッセージを読みました。すべて1つのメッセージに打ち込んでくれない？
What is Globalization?
Question: The world has become a globalized society in the 21st century.
What exactly is globalization?
Answer: The future dream of an American child is to be a soccer player,
the future dream of an Italian child is to be a manga artist,
and the future dream of a Japanese child is to be a major league baseball player.


## 留意事項 Attention
**Do not save this adapter merged with the base model**, as there exists a bug that reduces performance when saving this adapter merged with the model.    

このアダプターをモデルとマージして保存すると性能が下がってしまう不具合が存在するため、**ベースモデルとマージして保存しないでください**

## 謝辞 Acknowledgment

### Referenced models.
Original Base Model  
google/gemma-7b  
https://huggingface.co/google/gemma-7b

Base Model  
unsloth/gemma-7b-bnb-4bit  
https://huggingface.co/unsloth/gemma-7b-bnb-4bit

QLoRA Adapter  
webbigdata/C3TR-Adapter  
https://huggingface.co/webbigdata/C3TR-Adapter


This adapter was trained with Unsloth.  
https://github.com/unslothai/unsloth  

その他、[ALMA](https://arxiv.org/abs/2309.11674)をはじめ、コミュニティの皆さんからヒントを貰っています。ありがとう  
Other tips I have received from [ALMA](https://arxiv.org/abs/2309.11674) and others in the community. Thank you.  

このスクリプトは[webbigdata](https://webbigdata.jp/)によって作成されました  
This script was created by [webbigdata](https://webbigdata.jp/).  